# 類似度スコア算出

In [1]:
from gensim import models
w2v_model =  models.KeyedVectors.load_word2vec_format('jawiki.word_vectors.200d.txt', binary=False)

In [2]:
'''
# fugashiのインストール
%pip install fugashi
# gensimのインストール
%pip install gensim

# gensimのKeyedVectorsで使用されているword2vecモデルをダウンロード
# jawikiの200次元のword2vecモデル
wget https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.word_vectors.200d.txt.bz2
bunzip2 jawiki.word_vectors.200d.txt.bz2

# MeCabとipadicの辞書のインストール
# MeCabのインストール（Linuxの場合）
%sudo apt-get update
%sudo apt-get install mecab
%sudo apt-get install mecab libmecab-dev mecab-ipadic-utf8
# MeCabのインストール（macOSの場合、Homebrewを使用）
%brew install mecab
%brew install mecab mecab-ipadic

# numpyのインストール
%pip install numpy
'''

import ipadic
from fugashi import GenericTagger
import numpy as np
from numpy import dot
from numpy.linalg import norm
fugger = GenericTagger(ipadic.MECAB_ARGS)

# from gensim import models
# w2v_model =  models.KeyedVectors.load_word2vec_format('jawiki.word_vectors.200d.txt', binary=False)

def get_sim(s1, s2):
    sentence1 = [w.surface for w in fugger(s1)]
    sentence2 = [w.surface for w in fugger(s2)]
    if len(sentence1) > 0:
        for word in sentence1:
            if word == sentence1[0]:
                sentence_vec1 = w2v_model.__getitem__(word)
            else:
                sentence_vec1 = sentence_vec1 + w2v_model.__getitem__(word)
        sentence_vec1 = sentence_vec1 / len(sentence1)
    if len(sentence2) > 0:
        for word in sentence2:
            if word == sentence2[0]:
                sentence_vec2 = w2v_model.__getitem__(word)
            else:
                sentence_vec2 = sentence_vec2 + w2v_model.__getitem__(word)
        sentence_vec2 = sentence_vec2 / len(sentence2)
    print(f'Similarity Score = {dot(sentence_vec1, sentence_vec2)/(norm(sentence_vec1)*norm(sentence_vec2))}')

get_sim("エピソード１","エピソード２")

Similarity Score = 0.9893434047698975


In [6]:
import ipadic
from fugashi import GenericTagger
import numpy as np
from numpy import dot
from numpy.linalg import norm
fugger = GenericTagger(ipadic.MECAB_ARGS)

In [7]:
import pandas as pd

def get_sim2(s1, s2):
    sentence1 = [w.surface for w in fugger(s1) if w.surface in w2v_model]
    sentence2 = [w.surface for w in fugger(s2) if w.surface in w2v_model]
    if len(sentence1) > 0:
        for word in sentence1:
            if word == sentence1[0]:
                sentence_vec1 = w2v_model.__getitem__(word)
            else:
                sentence_vec1 = sentence_vec1 + w2v_model.__getitem__(word)
        sentence_vec1 = sentence_vec1 / len(sentence1)
    if len(sentence2) > 0:
        for word in sentence2:
            if word == sentence2[0]:
                sentence_vec2 = w2v_model.__getitem__(word)
            else:
                sentence_vec2 = sentence_vec2 + w2v_model.__getitem__(word)
        sentence_vec2 = sentence_vec2 / len(sentence2)
    similarity = dot(sentence_vec1, sentence_vec2)/(norm(sentence_vec1)*norm(sentence_vec2))
    return similarity

# Read the CSV file
df = pd.read_csv('data/logepisode.csv')
df['エピソード本文'] = df['エピソード本文'].str.replace('\r', '').str.replace('\n', '')

x = 10 # 比較したいエピソード番号
print(f'比較したいエピソード: Episode {x}')
print(f'Episode: {df['エピソード本文'][x]}')
print('上位3つの類似エピソード')
similarity_scores = []
for i in range(1, len(df['エピソード本文'])):
    if i != x:
        similarity = get_sim2(df['エピソード本文'][x], df['エピソード本文'][i])
        similarity_scores.append((i, similarity))

# Sort the similarity scores in descending order
similarity_scores.sort(key=lambda x: x[1], reverse=True)

# Extract the top 3 episodes
top_3_episodes = similarity_scores[:3]
# print(top_3_episodes)

# Print the top 3 episodes
for episode in top_3_episodes:
    episode_index = episode[0]
    similarity_score = episode[1]
    print(f"Episode {episode_index}: Similarity Score = {similarity_score}")
    print(df['エピソード本文'][episode_index])

比較したいエピソード: Episode 10
Episode: 生理期間中によそのお宅を訪問する予定が重なった時、その家のお手洗いを借りた時に生理用品を交換したくても、使用後の生理用品の対処が分かりません。みなさんどのようにされているのでしょうか？よその家に使用後のものを捨てさせてもらうのは恥ずかしいし申し訳なくてできません。かといってバッグに入れて持ち帰るのも、衛生的でない気がして躊躇してしまいます。なるべく、そのお宅に入る前後で交換して、お宅にいる間は交換しなくていいようにしていますが、泊まる予定だと長時間交換しないわけにもいきません。生理と訪問/宿泊が重ならないようにカレンダーとにらめっこをして予定を組みますが、予定外だとどうしようもありません。よそのお宅に行くときに生理が重なった時、他の人がどのようにしているのか知りたいです…！
上位3つの類似エピソード
Episode 7: Similarity Score = 0.9848636984825134
普段の家事分担の中で、買い物を担当することが多いです。ある日、パートナーから「おりものシートもついでにお願いしてもいい？」と何だか申し訳なさそうにお願いされました。売り場に入りにくいからでしょうか？それとも、どこかタブーのように感じてしまっているのでしょうか？私から理由を聞くのもはばかられたので、「全然いいよ！」と返すにとどまったのですが、、、。パートナーはどのような心理だったのでしょうか？？また、皆さんはこういった場面でどのように対応されているのでしょうか？？
Episode 23: Similarity Score = 0.9828354716300964
大学生のころ、友だちの家に遊びに行ったときに急に生理が始まった時、「いろいろあるよ！どれにする？！」とニコニコしながら大きな箱からたくさんの生理用品を出して種類とかについて説明してくれた時は、変に気後れしなくてよくなって、すごくうれしかったなぁと記憶に残ってます。
Episode 37: Similarity Score = 0.9822352528572083
メンタル的に多分、30代半ばから参ってはいたと、振り返れば思います。四季折々、いろいろな仲間とアウトドアや旅行や企画のない週末は無かったり、楽しさとワクワクを求めて元気いっぱい過ごしてはいました

## 改　Sentence BERT

In [3]:
# 必要なパッケージのインストール
# %pip install sentence-transformers pandas

from sentence_transformers import SentenceTransformer, util
import pandas as pd

# モデルのロード
model = SentenceTransformer('sonoisa/sentence-bert-base-ja-mean-tokens-v2')

# CSVファイルの読み込み
df = pd.read_csv('data/logepisode.csv')
df['エピソード本文'] = df['エピソード本文'].str.replace('\r', '').str.replace('\n', '')

# エピソードのベクトル化
embeddings = model.encode(df['エピソード本文'].tolist())

def get_sim2(s1_index, s2_index):
    vec1 = embeddings[s1_index]
    vec2 = embeddings[s2_index]
    similarity = util.pytorch_cos_sim(vec1, vec2).item()
    return similarity

x = 10  # 比較したいエピソード番号
print(f'比較したいエピソード: Episode {x}')
print(f'Episode: {df["エピソード本文"][x]}')
print('上位3つの類似エピソード')

similarity_scores = []
for i in range(len(df['エピソード本文'])):
    if i != x:
        similarity = get_sim2(x, i)
        similarity_scores.append((i, similarity))

# 類似度の高い順にソート
similarity_scores.sort(key=lambda x: x[1], reverse=True)

# 上位3つのエピソードを取得
top_3_episodes = similarity_scores[:3]

# 上位3つのエピソードを表示
for episode in top_3_episodes:
    episode_index = episode[0]
    similarity_score = episode[1]
    print(f"Episode {episode_index}: Similarity Score = {similarity_score}")
    print(df['エピソード本文'][episode_index])

比較したいエピソード: Episode 10
Episode: 生理期間中によそのお宅を訪問する予定が重なった時、その家のお手洗いを借りた時に生理用品を交換したくても、使用後の生理用品の対処が分かりません。みなさんどのようにされているのでしょうか？よその家に使用後のものを捨てさせてもらうのは恥ずかしいし申し訳なくてできません。かといってバッグに入れて持ち帰るのも、衛生的でない気がして躊躇してしまいます。なるべく、そのお宅に入る前後で交換して、お宅にいる間は交換しなくていいようにしていますが、泊まる予定だと長時間交換しないわけにもいきません。生理と訪問/宿泊が重ならないようにカレンダーとにらめっこをして予定を組みますが、予定外だとどうしようもありません。よそのお宅に行くときに生理が重なった時、他の人がどのようにしているのか知りたいです…！
上位3つの類似エピソード
Episode 36: Similarity Score = 0.6879069805145264
自分自身が違う環境に行ったわけではないのですが，職場の変化で驚いたのは，お手洗いに生理用品が置かれるようになったことです．自分で持ってきているのでそんなに共有の用品は利用しないかな？と思っていましたが，突然始まった時などにすごく助かってます．ただ，自分が高校生だと取りに行くのが恥ずかしくなるので共有スペースよりも個室に置いて欲しいと思うだろうなあと思いました（防犯面とかあるから難しいところですね）．
Episode 24: Similarity Score = 0.5941427946090698
大学生のころ、友だちの家に遊びに行ったときに急に生理が始まった時、申し訳ないなぁと思ってました。でも友だちが、「いろいろあるよ！どれにする？！使って使って！」とニコニコしながら大きな箱からたくさんの生理用品を出して種類とかについて説明してくれた時は、変に気後れしなくてよくなって、すごくうれしかったなぁと記憶に残ってます。
Episode 30: Similarity Score = 0.5914623737335205
最近，「生理用品の音が鳴るのがはずかしくて学校で生理用品の交換をためらう子がいる」という内容の記事を見ました．たしかに，「恥ずかしがらなくてもいいよ！」と思う派の自分も，廊下に音が聞こえるようなトイ